In [1]:
import re
import pandas as pd
import datetime
import mysql.connector


In [43]:
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [515]:
def LerFilename(revista):
    comando = f'select * from rpis_lidas where rpi="{revista}"'
    cursor.execute(comando)
    resultado = cursor.fetchall()
    #print(len(resultado))
    for row in resultado:
        #print(row[0])
        data_revista = row[1]
    df = pd.DataFrame(resultado)
    data = df[1].astype("string")
    str_data_revista = str(data_revista)
    d=datetime.datetime.strptime(str_data_revista, "%Y-%m-%d")
    #print(d.year)
    aux = str(d.strftime("%d%m%Y"))
    filename = f'revistas/Contratos_{revista}_{aux}.xml'
    return filename, str_data_revista

revista = 2708
filename, data_revista = LerFilename(revista)
print(filename)
print(data_revista)

revistas/Contratos_2708_29112022.xml
2022-11-29


In [516]:
def ConverteNumero(x):
    s = str(x)
    s = s.replace("BR","")
    s = s.replace(" ","")
    pos = s.find('-')
    return s[0:pos]

In [517]:
def ConverteData(data):
    d=datetime.datetime.strptime(data, "%d/%m/%Y")
    return f'{d.year:04}-{d.month:02}-{d.day:02}'

In [518]:
def LerRPI(filename,data_revista):
    file = open(file=filename,mode='r',encoding="utf8")
    file_content_raw=file.read()
    text1=re.compile("<despacho>")
    file_content=text1.split(file_content_raw)
    file.close()
    texto=file_content[0]
    t = re.compile(r'\d{2}/../\d{4}')
    check = t.findall(texto)
    data = ''.join(check)
    data=ConverteData(data)
    if data_revista == data:
        print("Data da RPI "+data_revista+" confere com a data gravada dentro do XML "+data) 
    else:
        print("Data da RPI "+data_revista+" não confere com a data gravada dentro do XML "+data) 
    return file_content

file_content = LerRPI(filename,data_revista)
print("Total registros: "+str(len(file_content)-1))

Data da RPI 2022-11-29 confere com a data gravada dentro do XML 2022-11-29
Total registros: 95


In [519]:
codigo_tag = re.compile('<codigo>(.*?)</codigo>')
titulo_tag = re.compile('<titulo>(.*?)</titulo>')
data_protocolo_tag = re.compile("<dataProtocolo inid=\"De\">(.*?)</dataProtocolo>") 
nome_completo_ce_tag = re.compile("<nomeCompleto inid=\"Ce\">(.*?)</nome-completo>") 
nome_completo_cs_tag = re.compile("<nomeCompleto inid=\"Cs\">(.*?)</nome-completo>") 
nome_completo_rq_tag = re.compile("<nomeCompleto inid=\"Rq\">(.*?)</nome-completo>") 

numero_Np_tag = re.compile("<numero inid=\"Np\">(.*?)</numero>") 
numero_Pt_tag = re.compile("<numero inid=\"Pt\">(.*?)</numero>") 
numero_Ca_tag = re.compile("<numero inid=\"Ca\">(.*?)</numero>") 

codigo_list, titulo_list, numero_peticao_list, requerente_list, data_protocolo_list, numero_list, cedente_nome_list, cessionaria_nome_list, pais_cedente_list, pais_cessionaria_list, cessionaria_setor_list, numero_ompi_list, data_ompi_list, classe_internacional_list, classe_nacional_list, sigla_prioridade_list, numero_prioridade_list, data_prioridade_list, prioridade_list, prioridade_interna_list, data_fase_nacional_list, titulo_pedido_list, inventor_nome_completo_list, divisao_pedido_list, data_divisao_pedido_list, data_rpi_list, data_concessao_list, numero_certificado_list, naturezaDocumento_list, textoObjeto_list, siglaCategoria_list, descricaoMoeda_list, valorContrato_list, prazoContrato_list, prazoVigenciaPI_list, observacao_list = ([] for i in range(36))

for line in file_content:
    
    codigo=codigo_tag.findall(line)
    titulo=titulo_tag.findall(line)

            
    numero_certificado=""
    naturezaDocumento=""
    textoObjeto=""
    siglaCategoria=""
    descricaoMoeda=""
    valorContrato=""
    prazoContrato=""
    prazoVigenciaPI=""
    observacao=""
    
    data_deposito=None
    numero=None
    nome_completo=None
    titulo_pedido=""
    data_rpi=" "
    data_protocolo = ""
    numero_peticao = ""
    requerente = ""
    cedente_nome = ""
    cessionaria_nome = ""
    pais_cedente = ""
    pais_cessionaria = ""
    cessionaria_setor = ""
    
    processos=re.findall("<processo-contrato>[\s\S]*</processo-contrato>",line)
    for processo in processos:
        data_protocolo=data_protocolo_tag.findall(processo)
        data_protocolo=', '.join(data_protocolo) # converte lista em string
        numero=numero_Np_tag.findall(processo)
        numero = ', '.join(numero)

        #peticoes
        peticoes_lista=re.findall("<peticoes>[\s\S]*</peticoes>",processo)
        for peticao_lista in peticoes_lista:
            peticoes=re.findall("<peticao>[\s\S]*</peticao>",peticao_lista)
            for peticao in peticoes:
                numeros=re.findall("<numero inid=\"Pt\">[\s\S]*</numero>",peticao)
                for numero_peticao in numeros:
                    n=re.findall("<numero inid=\"Pt\">(.*?)</numero>",numero_peticao)
                    numero_peticao = '; '.join(n)

                requerentes=re.findall("<requerente>[\s\S]*</requerente>",peticao)
                for r in requerentes:
                    n=re.findall("<nomeCompleto inid=\"Rq\">(.*?)</nomeCompleto>",r)
                    requerente = '; '.join(n)

        #cedentes
        cedentes_lista=re.findall("<cedentes>[\s\S]*</cedentes>",processo)
        for cedente_lista in cedentes_lista:
            cedentes=re.findall("<cedente>[\s\S]*</cedente>",cedente_lista)
            for cedente in cedentes:
                #print(cedente)
                #break
                cedente_nome=re.findall("<nomeCompleto inid=\"Ce\">(.*?)</nomeCompleto>",cedente)
                cedente_nome = '; '.join(cedente_nome)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",cedente)
                for endereco in enderecos:
                    #print(endereco)
                    #break
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        #print(pais)
                        #break
                        pais_cedente=re.findall("<nome inid=\"Pe\">(.*?)</nome>",pais)
                        pais_cedente = '; '.join(pais_cedente)

        #cessionarias
        cessionarias_lista=re.findall("<cessionarias>[\s\S]*</cessionarias>",processo)
        for cessionaria_lista in cessionarias_lista:
            cessionarias=re.findall("<cessionaria>[\s\S]*</cessionaria>",cessionaria_lista)
            for cessionaria in cessionarias:
                cessionaria_nome=re.findall("<nomeCompleto inid=\"Cs\">(.*?)</nomeCompleto>",cessionaria)
                cessionaria_nome = '; '.join(cessionaria_nome)
                cessionaria_setor=re.findall("<setor inid=\"Se\">(.*?)</setor>",cessionaria)
                cessionaria_setor = '; '.join(cessionaria_setor)
                enderecos=re.findall("<endereco>[\s\S]*</endereco>",cessionaria)
                for endereco in enderecos:
                    #print(endereco)
                    #break
                    paises=re.findall("<pais>[\s\S]*</pais>",endereco)
                    for pais in paises:
                        #print(pais)
                        #break
                        pais_cessionaria=re.findall("<nome inid=\"Ps\">(.*?)</nome>",pais)
                        pais_cessionaria = '; '.join(pais_cessionaria)

        #certificados
        certificados_lista=re.findall("<certificados>[\s\S]*</certificados>",processo)
        for certificado_lista in certificados_lista:
            certificados=re.findall("<certificado>[\s\S]*</certificado>",certificado_lista)
            for certificado in certificados:
                numero_certificado=re.findall("<numero inid=\"Ca\">(.*?)</numero>",certificado)
                numero_certificado = '; '.join(numero_certificado)
                naturezaDocumento=re.findall("<naturezaDocumento inid=\"Nd\">(.*?)</naturezaDocumento>",certificado)
                naturezaDocumento = '; '.join(naturezaDocumento)
                textoObjeto=re.findall("<textoObjeto inid=\"Ob\">(.*?)</textoObjeto>",certificado)
                textoObjeto = '; '.join(textoObjeto)
                siglaCategoria=re.findall("<siglaCategoria inid=\"Mc\">(.*?)</siglaCategoria>",certificado)
                siglaCategoria = '; '.join(siglaCategoria)
                descricaoMoeda=re.findall("<descricaoMoeda inid=\"Mo\">(.*?)</descricaoMoeda>",certificado)
                descricaoMoeda = '; '.join(descricaoMoeda)
                valorContrato=re.findall("<valorContrato inid=\"Va\">(.*?)</valorContrato>",certificado)
                valorContrato = '; '.join(valorContrato)
                prazoContrato=re.findall("<prazoContrato inid=\"Pz\">(.*?)</prazoContrato>",certificado)
                prazoContrato = '; '.join(prazoContrato)
                prazoVigenciaPI=re.findall("<prazoVigenciaPI inid=\"Pv\">(.*?)</prazoVigenciaPI>",certificado)
                prazoVigenciaPI = '; '.join(prazoVigenciaPI)
                observacao=re.findall("<observacao inid=\"Oc\">(.*?)</observacao>",certificado)
                observacao = '; '.join(observacao)
                        
    # checking length of gid is not equal to 0 then do append to all the lists
    if len(codigo)!=0:                             
        codigo_list.append(codigo[0])
        titulo_list.append(titulo[0])
        data_protocolo_list.append(data_protocolo)
        numero_list.append(numero)
        requerente_list.append(requerente)
        numero_peticao_list.append(numero_peticao)
        data_rpi_list.append(data_rpi[0])
        cedente_nome_list.append(cedente_nome)
        pais_cedente_list.append(pais_cedente)
        cessionaria_nome_list.append(cessionaria_nome)
        pais_cessionaria_list.append(pais_cessionaria)
        cessionaria_setor_list.append(cessionaria_setor)
        numero_certificado_list.append(numero_certificado)
        naturezaDocumento_list.append(naturezaDocumento)
        textoObjeto_list.append(textoObjeto)
        siglaCategoria_list.append(siglaCategoria)
        descricaoMoeda_list.append(descricaoMoeda)
        valorContrato_list.append(valorContrato)
        prazoContrato_list.append(prazoContrato)
        prazoVigenciaPI_list.append(prazoVigenciaPI)
        observacao_list.append(observacao)
        


In [520]:
data_frame = pd.DataFrame(
    {'despacho': codigo_list,
     'numero': numero_list,
     'data_protocolo': data_protocolo_list,
     'requerente': requerente_list,
     'numero_peticao': numero_peticao_list,
     'texto_despacho': titulo_list,
     'cedente': cedente_nome_list,
     'pais_cedente':pais_cedente_list,
     'cessionaria': cessionaria_nome_list,
     'pais_cessionaria': pais_cessionaria_list,
     'setor': cessionaria_setor_list,
     'certificado': numero_certificado_list,
     'naturezaDocumento': naturezaDocumento_list,
     'textoObjeto': textoObjeto_list,
     'siglaCategoria': siglaCategoria_list,
     'descricaoMoeda': descricaoMoeda_list,
     'valorContrato': valorContrato_list,
     'prazoContrato': prazoContrato_list,
     'prazoVigenciaPI': prazoVigenciaPI_list,
     'observacao': observacao_list
    })

In [521]:
data_frame["numero_new"] = data_frame["numero"].apply(ConverteNumero)
# https://pt.stackoverflow.com/questions/564421/aplicar-fun%c3%a7%c3%a3o-lamba-para-uma-subtra%c3%a7%c3%a3o-e-soma-python/564425#564425

In [522]:
print(len(data_frame.index))
data_frame[["despacho","numero_new","data_protocolo","requerente","numero_peticao","texto_despacho","cedente","pais_cedente","cessionaria","pais_cessionaria","setor","certificado","naturezaDocumento","textoObjeto","siglaCategoria","descricaoMoeda","valorContrato","prazoContrato","prazoVigenciaPI","observacao"]]
#df1 = data_frame.loc[data_frame['numero_new']=='702014000030']
#df1.head(1)
#data_frame.loc[89]
data_frame.head(1)

95


,despacho,numero,data_protocolo,requerente,numero_peticao,texto_despacho,cedente,pais_cedente,cessionaria,pais_cessionaria,...,certificado,naturezaDocumento,textoObjeto,siglaCategoria,descricaoMoeda,valorContrato,prazoContrato,prazoVigenciaPI,observacao,numero_new
0,135,BR 70 2020 000040-6,,ELETRONUCLEAR S.A.,880220002261,Petição indeferida,,,,,...,,,,,,,,,,702020000040


In [523]:
for i, infos in data_frame.iterrows():
    despacho = infos.despacho
    numero = infos.numero_new
    data_protocolo = infos.data_protocolo
    if data_protocolo is not None and data_protocolo!='':
        data_protocolo = ConverteData(infos.data_protocolo)
    else:
        data_protocolo = None
    requerente = infos.requerente
    requerente = requerente.replace("'", "\\'")
    numero_peticao = infos.numero_peticao
    texto_despacho = infos.texto_despacho
    cedente = infos.cedente
    cedente = cedente.replace("'", "\\'")
    pais_cedente = infos.pais_cedente
    cessionaria = infos.cessionaria
    cessionaria = cessionaria.replace("'", "\\'")
    pais_cessionaria = infos.pais_cessionaria
    setor = infos.setor
    setor = setor.replace("'", "\\'")
    certificado = infos.certificado
    naturezaDocumento = infos.naturezaDocumento
    textoObjeto = infos.textoObjeto
    textoObjeto = textoObjeto.replace("'", "\\'")
    siglaCategoria = infos.siglaCategoria
    descricaoMoeda = infos.descricaoMoeda
    valorContrato = infos.valorContrato
    prazoContrato = infos.prazoContrato
    prazoVigenciaPI = infos.prazoVigenciaPI
    observacao = infos.observacao
    observacao = observacao.replace("'", "\\'")
   
    comando = f"INSERT INTO arquivadosct (id, data, despacho, numero, data_protocolo, requerente, numero_peticao, texto_despacho, cedente, pais_cedente, cessionaria, pais_cessionaria, setor, certificado, naturezaDocumento, textoObjeto, siglaCategoria, descricaoMoeda, valorContrato, prazoContrato, prazoVigenciaPI, observacao) VALUES (NULL, '{data_revista}', '{despacho}', '{numero}', '{data_protocolo}', '{requerente}', '{numero_peticao}','{texto_despacho}', '{cedente}', '{pais_cedente}', '{cessionaria}', '{pais_cessionaria}', '{setor}', '{certificado}', '{naturezaDocumento}', '{textoObjeto}', '{siglaCategoria}','{descricaoMoeda}', '{valorContrato}', '{prazoContrato}', '{prazoVigenciaPI}', '{observacao}');"
    if data_protocolo is None:
        comando = f"INSERT INTO arquivadosct (id, data, despacho, numero, data_protocolo, requerente, numero_peticao, texto_despacho, cedente, pais_cedente, cessionaria, pais_cessionaria, setor, certificado, naturezaDocumento, textoObjeto, siglaCategoria, descricaoMoeda, valorContrato, prazoContrato, prazoVigenciaPI, observacao) VALUES (NULL, '{data_revista}', '{despacho}', '{numero}', NULL, '{requerente}', '{numero_peticao}','{texto_despacho}', '{cedente}', '{pais_cedente}', '{cessionaria}', '{pais_cessionaria}', '{setor}', '{certificado}', '{naturezaDocumento}', '{textoObjeto}', '{siglaCategoria}','{descricaoMoeda}', '{valorContrato}', '{prazoContrato}', '{prazoVigenciaPI}', '{observacao}');"

    #print(comando)
    #break
    try:
        cursor.execute(comando)
        conexao.commit()
    except (mysql.connector.Error, mysql.connector.Warning) as e:
        print(e)
        print(str(numero))

print("Comando encerrado!")

Comando encerrado!


In [524]:
#confere total de registros gravados
comando = f'select count(*) as total from arquivadosct where data="{data_revista}"'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    if len(data_frame.index) == row[0]:
        print("Sucesso na gravação do total de "+str(row[0])+" registros !")
    else:
        print("Total de registros gravados: "+str(row[0])+" registros, quando toral era de "+len(data_frame.index))



Sucesso na gravação do total de 95 registros !


In [540]:
def numOfDays(date1, date2):
    return (date2-date1).days

#confere média de tempo de registro
soma = 0
total = 0
comando = f'select * from arquivadosct where despacho="350" and year(data)=2022'
cursor.execute(comando)
resultado = cursor.fetchall()
for row in resultado:
    print(row[3])
    print("data concessão= "+str(row[1]))
    d=datetime.datetime.strptime(str(row[1]), "%Y-%m-%d")
    date1 = datetime.date(d.year, d.month, d.day)
    print("data protocolo= "+str(row[4]))
    d=datetime.datetime.strptime(str(row[4]), "%Y-%m-%d")
    date2 = datetime.date(d.year, d.month, d.day)
    print(numOfDays(date2, date1), "dias")
    soma = soma + numOfDays(date2, date1)
    total = total + 1
    print("\n")
    
print("Fim de processamento")
media = soma/total
print(f'Tempo médio de dias para concessão: {media:.2f}')


07026
data concessão= 2022-01-04
data protocolo= 2021-12-10
25 dias


11067
data concessão= 2022-01-04
data protocolo= 2021-02-12
326 dias


11119
data concessão= 2022-01-04
data protocolo= 2021-05-14
235 dias


12099
data concessão= 2022-01-04
data protocolo= 2021-12-09
26 dias


702015000285
data concessão= 2022-01-04
data protocolo= 2021-11-04
61 dias


702017000024
data concessão= 2022-01-04
data protocolo= 2021-12-21
14 dias


702017000221
data concessão= 2022-01-04
data protocolo= 2021-12-03
32 dias


702018000057
data concessão= 2022-01-04
data protocolo= 2021-12-03
32 dias


702018000058
data concessão= 2022-01-04
data protocolo= 2021-12-03
32 dias


702019000026
data concessão= 2022-01-04
data protocolo= 2021-12-03
32 dias


702019000654
data concessão= 2022-01-04
data protocolo= 2021-12-10
25 dias


702020000491
data concessão= 2022-01-04
data protocolo= 2021-12-20
15 dias


702021000228
data concessão= 2022-01-04
data protocolo= 2021-05-17
232 dias


702021000387
data conces

98 dias


702022000049
data concessão= 2022-04-05
data protocolo= 2022-01-17
78 dias


702022000109
data concessão= 2022-04-05
data protocolo= 2022-02-11
53 dias


702022000130
data concessão= 2022-04-05
data protocolo= 2022-02-17
47 dias


702022000167
data concessão= 2022-04-05
data protocolo= 2022-03-17
19 dias


702022000172
data concessão= 2022-04-05
data protocolo= 2022-03-18
18 dias


702022000173
data concessão= 2022-04-05
data protocolo= 2022-03-18
18 dias


93107
data concessão= 2022-04-05
data protocolo= 2022-03-17
19 dias


09028
data concessão= 2022-04-12
data protocolo= 2022-03-31
12 dias


10030
data concessão= 2022-04-12
data protocolo= 2022-03-30
13 dias


13066
data concessão= 2022-04-12
data protocolo= 2022-03-29
14 dias


14032
data concessão= 2022-04-12
data protocolo= 2022-03-18
25 dias


702014000438
data concessão= 2022-04-12
data protocolo= 2022-03-31
12 dias


702017000418
data concessão= 2022-04-12
data protocolo= 2021-11-19
144 dias


702020000528
data conce

244 dias


702022000075
data concessão= 2022-08-16
data protocolo= 2022-08-02
14 dias


702022000259
data concessão= 2022-08-16
data protocolo= 2022-05-13
95 dias


702022000266
data concessão= 2022-08-16
data protocolo= 2022-05-17
91 dias


702022000289
data concessão= 2022-08-16
data protocolo= 2022-05-25
83 dias


702022000396
data concessão= 2022-08-16
data protocolo= 2022-07-15
32 dias


702022000412
data concessão= 2022-08-16
data protocolo= 2022-07-22
25 dias


702022000413
data concessão= 2022-08-16
data protocolo= 2022-07-22
25 dias


702022000414
data concessão= 2022-08-16
data protocolo= 2022-07-22
25 dias


702022000415
data concessão= 2022-08-16
data protocolo= 2022-07-25
22 dias


702022000416
data concessão= 2022-08-16
data protocolo= 2022-07-25
22 dias


702022000423
data concessão= 2022-08-16
data protocolo= 2022-07-28
19 dias


702022000428
data concessão= 2022-08-16
data protocolo= 2022-07-30
17 dias


83247
data concessão= 2022-08-16
data protocolo= 2022-07-28
19 di

Processamento encerrado


520


'1.3'

In [ ]:
CREATE TABLE `arquivadosct` ( `id` int(11) NULL AUTO_INCREMENT, `despacho` char(30) NULL, `numero` char(15) NULL, `data_protocolo` date NULL, `requerente` TEXT NULL, `numero_peticao` char(15) NULL, `texto_despacho` TEXT NULL, `cedente` TEXT NULL, `pais_cedente` char(30) NULL, `cessionaria` TEXT NULL, `pais_cessionaria` char(30) NULL, `setor` TEXT NULL, `certificado` char(30) NULL, `naturezaDocumento` char(30) NULL, `textoObjeto` TEXT NULL, `siglaCategoria` char(30) NULL, `descricaoMoeda` char(15) NULL, `valorContrato` char(255) NULL, `prazoContrato` char(255) NULL, `prazoVigenciaPI` TEXT NULL, `observacao` TEXT NULL ) ENGINE=InnoDB DEFAULT CHARSET=latin1 COMMENT='todos os despachos publicados nas RPIs'; 
ALTER TABLE `arquivadosct` ADD PRIMARY KEY (`id`); ALTER TABLE `arquivadosxml` MODIFY `id` int(11) NOT NULL AUTO_INCREMENT; 